<a href="https://colab.research.google.com/github/Natsumisnooker/Machine-Learning/blob/main/ML_Reproduced_and_Additional_Feature_and_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import io
import os
from datetime import datetime

# ติดตั้ง required packages อัตโนมัติ
print("📦 ติดตั้ง packages ที่จำเป็น...")
try:
    import xlsxwriter
    print("✅ xlsxwriter ติดตั้งแล้ว")
except ImportError:
    !pip install xlsxwriter
    import xlsxwriter

try:
    from textstat import flesch_reading_ease, smog_index, dale_chall_readability_score, coleman_liau_index, gunning_fog
    print("✅ textstat ติดตั้งแล้ว")
except ImportError:
    !pip install textstat
    from textstat import flesch_reading_ease, smog_index, dale_chall_readability_score, coleman_liau_index, gunning_fog

# ติดตั้ง packages เพิ่มเติมสำหรับโมเดลใหม่
try:
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.linear_model import SGDClassifier
    from sklearn.ensemble import GradientBoostingClassifier
    print("✅ scikit-learn models ติดตั้งแล้ว")
except ImportError:
    !pip install scikit-learn --upgrade
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.linear_model import SGDClassifier
    from sklearn.ensemble import GradientBoostingClassifier

print("✅ ติดตั้ง packages เสร็จสิ้น")

# ดาวน์โหลดข้อมูล NLTK ที่จำเป็น
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

# ดาวน์โหลด punkt_tab เพิ่มเติมตามข้อผิดพลาด
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

class AdvancedStylometricFeatureExtractor:
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))
        self.imperative_verbs = ['click', 'verify', 'submit', 'download', 'update']
        self.modal_verbs = ['can', 'could', 'should', 'must', 'will', 'would']
        self.uncertainty_adverbs = ['maybe', 'possibly', 'perhaps']
        self.technical_jargon = ['security', 'account', 'update', 'technical', 'password', 'login']
        self.promotional_words = ['offer', 'deal', 'free', 'discount', 'limited', 'exclusive']
        self.politeness_markers = ['please', 'thank', 'appreciate', 'kindly']
        self.aggressiveness_markers = ['must', 'now', 'immediately']
        self.urgency_markers = ['urgent', 'asap', 'immediately', 'urgently']
        self.conditional_phrases = ['if', 'unless']
        self.personalization_markers = ['you', 'your']
        self.conjunctions = ['and', 'but', 'or', 'because']
        self.prepositions = ['in', 'on', 'at', 'by', 'with']
        self.function_words = ['the', 'is', 'at', 'which', 'on']
        self.pronouns = ['i', 'you', 'he', 'she', 'it', 'we', 'they']

        # เพิ่มคำศัพท์สำหรับ features ใหม่
        self.positive_emotion_words = ['great', 'good', 'excellent', 'amazing', 'wonderful', 'happy', 'nice']
        self.negative_emotion_words = ['bad', 'terrible', 'awful', 'horrible', 'sad', 'angry', 'frustrated']
        self.formal_words = ['sincerely', 'regards', 'respectfully', 'cordially', 'yours']
        self.informal_words = ['hey', 'hi', 'hello', 'thanks', 'cheers', 'bye']
        self.legal_terms = ['terms', 'conditions', 'agreement', 'policy', 'compliance', 'regulation']

    def count_syllables(self, word):
        """นับจำนวน syllables ในคำ"""
        word = word.lower()
        count = 0
        vowels = "aeiouy"
        if word[0] in vowels:
            count += 1
        for index in range(1, len(word)):
            if word[index] in vowels and word[index - 1] not in vowels:
                count += 1
        if word.endswith("e"):
            count -= 1
        if count == 0:
            count += 1
        return count

    def calculate_readability_scores(self, text):
        """คำนวณ readability scores ด้วย fallback ถ้า textstat ไม่ทำงาน"""
        try:
            flesch = flesch_reading_ease(text)
            smog = smog_index(text)
            dale_chall = dale_chall_readability_score(text)
            coleman_liau = coleman_liau_index(text)
            gunning_fog_score = gunning_fog(text)
        except:
            words = word_tokenize(text)
            sentences = sent_tokenize(text)

            if len(words) == 0 or len(sentences) == 0:
                return 0, 0, 0, 0, 0

            avg_sentence_length = len(words) / len(sentences)
            avg_syllables_per_word = np.mean([self.count_syllables(word) for word in words if word.isalnum()])

            flesch = max(0, min(100, 206.835 - 1.015 * avg_sentence_length - 84.6 * avg_syllables_per_word))
            smog = max(0, avg_sentence_length + 6)
            dale_chall = max(0, avg_sentence_length * 0.1579 + 3.6365)
            coleman_liau = max(0, avg_sentence_length * 0.0588 - 15.8)
            gunning_fog_score = max(0, avg_sentence_length * 0.4)

        return flesch, smog, dale_chall, coleman_liau, gunning_fog_score

    def extract_advanced_features(self, text):
        """Extract 5 advanced stylometric features ใหม่"""
        features = {}
        text_lower = text.lower()

        # 1. Emotional Tone Score (คะแนนโทนอารมณ์)
        positive_count = sum(text_lower.count(word) for word in self.positive_emotion_words)
        negative_count = sum(text_lower.count(word) for word in self.negative_emotion_words)
        total_emotion_words = positive_count + negative_count

        if total_emotion_words > 0:
            features['emotional_tone_score'] = (positive_count - negative_count) / total_emotion_words
        else:
            features['emotional_tone_score'] = 0

        # 2. Formality Index (ดัชนีความเป็นทางการ)
        formal_count = sum(text_lower.count(word) for word in self.formal_words)
        informal_count = sum(text_lower.count(word) for word in self.informal_words)
        total_formality_words = formal_count + informal_count

        if total_formality_words > 0:
            features['formality_index'] = formal_count / total_formality_words
        else:
            features['formality_index'] = 0.5  # ค่า neutral

        # 3. Legal Terminology Density (ความหนาแน่นของคำทางกฎหมาย)
        legal_count = sum(text_lower.count(word) for word in self.legal_terms)
        words = word_tokenize(text)
        words_clean = [word for word in words if word.isalnum()]

        features['legal_terminology_density'] = legal_count / len(words_clean) if words_clean else 0

        # 4. Sentence Structure Complexity (ความซับซ้อนของโครงสร้างประโยค)
        sentences = sent_tokenize(text)
        if sentences:
            # คำนวณความแปรปรวนของความยาวประโยค
            sentence_lengths = [len(word_tokenize(sent)) for sent in sentences]
            features['sentence_length_variation'] = np.std(sentence_lengths) if len(sentence_lengths) > 1 else 0
        else:
            features['sentence_length_variation'] = 0

        # 5. Question Mark Density (ความหนาแน่นของเครื่องหมายคำถาม)
        question_marks = text.count('?')
        features['question_mark_density'] = question_marks / len(words_clean) if words_clean else 0

        return features

    def extract_all_features(self, text):
        """Extract all stylometric features (65 features)"""
        features = {}

        # Basic text cleaning
        text = str(text)
        words = word_tokenize(text)
        sentences = sent_tokenize(text)

        # Filter out non-alphanumeric words for some features
        words_clean = [word for word in words if word.isalnum()]

        # 1. Lexical Features (16.7%)
        features['word_count'] = len(words_clean)
        features['character_count'] = len(text)
        features['average_word_length'] = np.mean([len(word) for word in words_clean]) if words_clean else 0
        features['sentence_count'] = len(sentences)
        features['average_sentence_length'] = len(words_clean) / len(sentences) if sentences else 0
        features['unique_word_count'] = len(set(words_clean))
        features['lexical_diversity'] = features['unique_word_count'] / features['word_count'] if features['word_count'] > 0 else 0
        features['number_of_emails'] = text.count('@')
        features['uppercase_word_count'] = sum(1 for word in words_clean if word.isupper())
        features['uppercase_word_ratio'] = features['uppercase_word_count'] / features['word_count'] if features['word_count'] > 0 else 0
        features['complex_words_count'] = sum(1 for word in words_clean if len(word) > 6)
        features['average_syllables_per_word'] = np.mean([self.count_syllables(word) for word in words_clean]) if words_clean else 0

        # 2. Syntactic Features (18.3%)
        features['comma_count'] = text.count(',')
        features['semicolon_count'] = text.count(';')
        features['colon_count'] = text.count(':')
        features['exclamation_count'] = text.count('!')
        features['quotation_count'] = text.count('"')
        features['dash_count'] = text.count('-')

        # Sentence complexity
        conjunction_count = sum(text.lower().count(conj) for conj in self.conjunctions)
        features['sentence_complexity_ratio'] = conjunction_count / features['sentence_count'] if features['sentence_count'] > 0 else 0
        features['clause_density'] = features['sentence_complexity_ratio'] / features['sentence_count'] if features['sentence_count'] > 0 else 0

        # Density features
        pronoun_count = sum(text.lower().count(pronoun) for pronoun in self.pronouns)
        features['pronoun_density'] = pronoun_count / features['word_count'] if features['word_count'] > 0 else 0

        preposition_count = sum(text.lower().count(prep) for prep in self.prepositions)
        features['preposition_density'] = preposition_count / features['word_count'] if features['word_count'] > 0 else 0

        function_word_count = sum(text.lower().count(func) for func in self.function_words)
        features['function_word_density'] = function_word_count / features['word_count'] if features['word_count'] > 0 else 0

        # 3. Punctuation Features (16.7%)
        punctuation_chars = [',', ';', ':', '!', '"', '-', '.', '?', "'", '(', ')', '[', ']', '{', '}']
        for char in punctuation_chars:
            features[f'punctuation_{char}_freq'] = text.count(char) / len(text) if len(text) > 0 else 0

        features['punctuation_variety'] = len(set(char for char in text if char in punctuation_chars))

        # 4. Readability Scores (16.7%)
        flesch, smog, dale_chall, coleman_liau, gunning_fog_score = self.calculate_readability_scores(text)
        features['flesch_reading_ease'] = flesch
        features['smog_index'] = smog
        features['dale_chall_score'] = dale_chall
        features['coleman_liau_index'] = coleman_liau
        features['gunning_fog_index'] = gunning_fog_score

        # 5. Word Category Features (11.7%)
        features['number_of_pronouns'] = pronoun_count
        features['first_person_pronoun_count'] = sum(text.lower().count(pronoun) for pronoun in ['i', 'we', 'my', 'our'])
        features['second_person_pronoun_count'] = sum(text.lower().count(pronoun) for pronoun in ['you', 'your'])
        features['imperative_verbs_count'] = sum(text.lower().count(verb) for verb in self.imperative_verbs)
        features['modal_verbs_count'] = sum(text.lower().count(verb) for verb in self.modal_verbs)
        features['uncertainty_adverbs_count'] = sum(text.lower().count(adverb) for adverb in self.uncertainty_adverbs)
        features['technical_jargon_count'] = sum(text.lower().count(jargon) for jargon in self.technical_jargon)
        features['promotional_words_count'] = sum(text.lower().count(word) for word in self.promotional_words)

        # 6. Email-Specific Features (6.7%)
        features['attachment_mentions'] = text.lower().count('attachment')

        # 7. Complexity Features (5.0%)
        vectorizer_bigram = CountVectorizer(ngram_range=(2, 2), max_features=100)
        vectorizer_trigram = CountVectorizer(ngram_range=(3, 3), max_features=100)

        try:
            bigram_matrix = vectorizer_bigram.fit_transform([text])
            trigram_matrix = vectorizer_trigram.fit_transform([text])
            features['bigram_count'] = bigram_matrix.sum()
            features['trigram_count'] = trigram_matrix.sum()
        except:
            features['bigram_count'] = 0
            features['trigram_count'] = 0

        word_lengths = [len(word) for word in words_clean]
        features['word_length_variation'] = np.std(word_lengths) if word_lengths else 0

        # 8. Stylistic Features (8.3%)
        features['politeness_markers_count'] = sum(text.lower().count(marker) for marker in self.politeness_markers)
        features['aggressiveness_markers_count'] = sum(text.lower().count(marker) for marker in self.aggressiveness_markers)
        features['urgency_markers_count'] = sum(text.lower().count(marker) for marker in self.urgency_markers)
        features['conditional_phrases_count'] = sum(text.lower().count(phrase) for phrase in self.conditional_phrases)
        features['personalization_markers_count'] = sum(text.lower().count(marker) for marker in self.personalization_markers)

        # 9. Advanced Features ใหม่ 5 ตัว
        advanced_features = self.extract_advanced_features(text)
        features.update(advanced_features)

        return features

def create_complete_excel_report(train_df, test_df, train_features_df, test_features_df,
                                results_df, feature_importance_df, ablation_df, model_name):
    """สร้างไฟล์ Excel รายงานละเอียดแบบครบถ้วน รวม Subject และ Body"""

    # สร้าง Excel writer
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'complete_stylometric_analysis_report_{timestamp}.xlsx'

    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:

        # 1. สรุปข้อมูลพื้นฐาน
        summary_data = {
            'รายการ': [
                'จำนวนข้อมูลฝึก',
                'จำนวนข้อมูลทดสอบ',
                'จำนวน Features',
                'วันที่วิเคราะห์',
                'โมเดลที่ดีที่สุด',
                'ความแม่นยำสูงสุด',
                'จำนวนโมเดลที่ทดสอบ'
            ],
            'ค่า': [
                len(train_df),
                len(test_df),
                train_features_df.shape[1],
                datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                model_name,
                f"{results_df.loc[model_name, 'Accuracy']:.4f}",
                len(results_df)
            ]
        }
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='สรุปผล', index=False)

        # 2. ข้อมูลต้นฉบับ + Features (Train)
        train_complete = train_df.copy()
        if 'combined_text' not in train_complete.columns:
            if 'Subject' in train_complete.columns and 'Body' in train_complete.columns:
                train_complete['combined_text'] = train_complete['Subject'] + " " + train_complete['Body']

        for feature in train_features_df.columns:
            train_complete[feature] = train_features_df[feature].values

        train_complete.to_excel(writer, sheet_name='ข้อมูลต้นฉบับ_Train', index=False)

        # 3. ข้อมูลต้นฉบับ + Features (Test)
        test_complete = test_df.copy()
        if 'combined_text' not in test_complete.columns:
            if 'Subject' in test_complete.columns and 'Body' in test_complete.columns:
                test_complete['combined_text'] = test_complete['Subject'] + " " + test_complete['Body']

        for feature in test_features_df.columns:
            test_complete[feature] = test_features_df[feature].values

        test_complete.to_excel(writer, sheet_name='ข้อมูลต้นฉบับ_Test', index=False)

        # 4. สถิติ Features
        train_stats = train_features_df.describe().T
        train_stats['feature'] = train_stats.index
        train_stats = train_stats[['feature', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
        train_stats.to_excel(writer, sheet_name='สถิติFeatures_Train', index=False)

        test_stats = test_features_df.describe().T
        test_stats['feature'] = test_stats.index
        test_stats = test_stats[['feature', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
        test_stats.to_excel(writer, sheet_name='สถิติFeatures_Test', index=False)

        # 5. ผลลัพธ์โมเดล
        results_df.to_excel(writer, sheet_name='ผลลัพธ์โมเดล', index=True)

        # 6. Feature Importance
        if feature_importance_df is not None:
            feature_importance_df.to_excel(writer, sheet_name='Feature_Importance', index=False)

        # 7. Comparison between Train and Test Features
        comparison_data = []
        for feature in train_features_df.columns:
            train_mean = train_features_df[feature].mean()
            test_mean = test_features_df[feature].mean()
            train_std = train_features_df[feature].std()
            test_std = test_features_df[feature].std()

            comparison_data.append({
                'Feature': feature,
                'Train_Mean': train_mean,
                'Test_Mean': test_mean,
                'Train_STD': train_std,
                'Test_STD': test_std,
                'Mean_Difference': abs(train_mean - test_mean),
                'STD_Difference': abs(train_std - test_std)
            })

        comparison_df = pd.DataFrame(comparison_data)
        comparison_df.to_excel(writer, sheet_name='เปรียบเทียบFeatures', index=False)

        # 8. ตัวอย่างอีเมล
        sample_train = train_complete.head().copy()
        sample_train.to_excel(writer, sheet_name='ตัวอย่างอีเมล_Train', index=False)

        sample_test = test_complete.head().copy()
        sample_test.to_excel(writer, sheet_name='ตัวอย่างอีเมล_Test', index=False)

    return filename

# อัพโหลดไฟล์
print("📁 กรุณาอัพโหลดไฟล์ Train.csv และ Test.csv")
uploaded = files.upload()

# แสดงชื่อไฟล์ที่อัพโหลดได้
print("\n📂 ไฟล์ที่อัพโหลด:")
for filename in uploaded.keys():
    print(f"  - {filename}")

# หาชื่อไฟล์ที่ถูกต้องโดยอัตโนมัติ
train_filename = None
test_filename = None

for filename in uploaded.keys():
    if 'train' in filename.lower():
        train_filename = filename
    elif 'test' in filename.lower():
        test_filename = filename

if not train_filename:
    train_filename = list(uploaded.keys())[0]
if not test_filename and len(uploaded.keys()) > 1:
    test_filename = list(uploaded.keys())[1]

print(f"\n🔍 ใช้ไฟล์:")
print(f"  Train: {train_filename}")
print(f"  Test: {test_filename}")

# โหลดข้อมูลจากไฟล์ที่อัพโหลด
train_df = pd.read_csv(io.BytesIO(uploaded[train_filename]))
test_df = pd.read_csv(io.BytesIO(uploaded[test_filename]))

# แสดงข้อมูลพื้นฐาน
print("\n" + "=" * 80)
print("📊 ข้อมูลพื้นฐานของ Dataset")
print("=" * 80)
print(f"Train dataset shape: {train_df.shape}")
print(f"Test dataset shape: {test_df.shape}")

print("\nคอลัมน์ใน Train dataset:")
print(train_df.columns.tolist())

# ตรวจสอบโครงสร้างข้อมูล
print("\n" + "=" * 80)
print("🔍 การตรวจสอบโครงสร้างข้อมูล")
print("=" * 80)

has_subject = 'Subject' in train_df.columns
has_body = 'Body' in train_df.columns
has_label = 'Label' in train_df.columns

print(f"มีคอลัมน์ Subject: {has_subject}")
print(f"มีคอลัมน์ Body: {has_body}")
print(f"มีคอลัมน์ Label: {has_label}")

# สร้างคอลัมน์ combined_text สำหรับการ extract features
if has_subject and has_body:
    train_df['combined_text'] = train_df['Subject'].astype(str) + " " + train_df['Body'].astype(str)
    test_df['combined_text'] = test_df['Subject'].astype(str) + " " + test_df['Body'].astype(str)
    print("✅ สร้าง combined_text จาก Subject และ Body")
else:
    text_column = train_df.columns[0]
    print(f"📝 ใช้คอลัมน์ '{text_column}' เป็นข้อความ")
    train_df['combined_text'] = train_df[text_column].astype(str)
    test_df['combined_text'] = test_df[text_column].astype(str)

# Extract Stylometric Features
print("\n" + "=" * 80)
print("🔧 การ Extract Stylometric Features (65 features)")
print("=" * 80)

feature_extractor = AdvancedStylometricFeatureExtractor()

print("กำลัง extract features จาก train dataset...")
train_features = []
for idx, text in enumerate(train_df['combined_text']):
    if idx % 10 == 0:
        print(f"Processed {idx}/{len(train_df)} emails...")
    features = feature_extractor.extract_all_features(text)
    train_features.append(features)

print("กำลัง extract features จาก test dataset...")
test_features = []
for idx, text in enumerate(test_df['combined_text']):
    if idx % 10 == 0:
        print(f"Processed {idx}/{len(test_df)} emails...")
    features = feature_extractor.extract_all_features(text)
    test_features.append(features)

# แปลงเป็น DataFrame
X_train = pd.DataFrame(train_features)
X_test = pd.DataFrame(test_features)

y_train = train_df['Label']
y_test = test_df['Label']

print(f"\n✅ การ extract features เสร็จสมบูรณ์")
print(f"จำนวน features ที่ได้: {X_train.shape[1]}")
print(f"จำนวนข้อมูลฝึก: {X_train.shape[0]}")
print(f"จำนวนข้อมูลทดสอบ: {X_test.shape[0]}")

# บันทึกไฟล์ CSV ของ Features
print("\n💾 กำลังบันทึกไฟล์ Features...")
X_train.to_csv('stylometric_features_train.csv', index=False)
X_test.to_csv('stylometric_features_test.csv', index=False)

X_train_with_label = X_train.copy()
X_train_with_label['Label'] = y_train.values
X_test_with_label = X_test.copy()
X_test_with_label['Label'] = y_test.values

X_train_with_label.to_csv('stylometric_features_train_with_labels.csv', index=False)
X_test_with_label.to_csv('stylometric_features_test_with_labels.csv', index=False)

print("✅ บันทึกไฟล์ Features เรียบร้อยแล้ว")

# เติมค่าที่หายไปด้วยค่าเฉลี่ย
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

# Encode target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

class_names = label_encoder.classes_
print(f"คลาส: {class_names}")

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# กำหนดโมเดลทั้งหมด 10 ตัว
models = {
    # โมเดลเดิม 4 ตัว
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'SVM': SVC(random_state=42, probability=True),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),

    # โมเดลใหม่ 6 ตัว
    'L1 Regularization (SGD)': SGDClassifier(loss='log_loss', penalty='l1', alpha=0.01, random_state=42),
    'L2 Regularization (SGD)': SGDClassifier(loss='log_loss', penalty='l2', alpha=0.01, random_state=42),
    'Elastic Net (SGD)': SGDClassifier(loss='log_loss', penalty='elasticnet', alpha=0.01, l1_ratio=0.5, random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

print(f"\n🔢 จำนวนโมเดลที่ทดสอบ: {len(models)} โมเดล")

# ฝึกและประเมินโมเดล
print("\n" + "=" * 80)
print("🤖 การฝึกและประเมินโมเดลทั้งหมด")
print("=" * 80)

results = {}
confusion_matrices = {}
trained_models = {}

for name, model in models.items():
    print(f"\n🎯 กำลังฝึกโมเดล: {name}")

    try:
        # ฝึกโมเดล
        model.fit(X_train_scaled, y_train_encoded)
        trained_models[name] = model

        # ทำนาย
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1] if hasattr(model, "predict_proba") else None

        # คำนวณเมตริก
        accuracy = accuracy_score(y_test_encoded, y_pred)
        precision = precision_score(y_test_encoded, y_pred, average='weighted')
        recall = recall_score(y_test_encoded, y_pred, average='weighted')
        f1 = f1_score(y_test_encoded, y_pred, average='weighted')

        if y_pred_proba is not None:
            auc = roc_auc_score(y_test_encoded, y_pred_proba)
        else:
            # สำหรับโมเดลที่ไม่มี predict_proba
            auc = 0.5

        # บันทึกผลลัพธ์
        results[name] = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1-Score': f1,
            'AUC-Score': auc
        }

        # บันทึก confusion matrix
        cm = confusion_matrix(y_test_encoded, y_pred)
        confusion_matrices[name] = cm

        print(f"✅ {name} Results:")
        print(f"   Accuracy:  {accuracy:.4f}")
        print(f"   Precision: {precision:.4f}")
        print(f"   Recall:    {recall:.4f}")
        print(f"   F1-Score:  {f1:.4f}")
        print(f"   AUC-Score: {auc:.4f}")

    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดในการฝึกโมเดล {name}: {e}")
        results[name] = {
            'Accuracy': 0,
            'Precision': 0,
            'Recall': 0,
            'F1-Score': 0,
            'AUC-Score': 0
        }

# แสดงผลลัพธ์ในรูปแบบตาราง
print("\n" + "=" * 80)
print("📈 สรุปผลลัพธ์ทั้งหมด")
print("=" * 80)
results_df = pd.DataFrame(results).T
results_df_sorted = results_df.sort_values('Accuracy', ascending=False)
print(results_df_sorted.round(4))

# บันทึกผลลัพธ์โมเดลเป็น CSV
results_df.to_csv('model_performance_results.csv', index=True)
print("✅ บันทึกผลลัพธ์โมเดลเป็น CSV เรียบร้อยแล้ว")

# หาโมเดลที่ดีที่สุด
best_model_name = results_df['Accuracy'].idxmax()
best_accuracy = results_df['Accuracy'].max()

# Feature Importance Analysis
print("\n" + "=" * 80)
print("🔍 การวิเคราะห์ Feature Importance")
print("=" * 80)

xgb_feature_importance_df = None
rf_feature_importance_df = None
gb_feature_importance_df = None

# XGBoost Feature Importance
if 'XGBoost' in trained_models:
    xgb_model = trained_models['XGBoost']
    xgb_importance = xgb_model.feature_importances_

    xgb_feature_importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': xgb_importance
    }).sort_values('Importance', ascending=False)

    print("\n📊 Top 15 Feature Importances (XGBoost):")
    print(xgb_feature_importance_df.head(15))
    xgb_feature_importance_df.to_csv('xgb_feature_importance.csv', index=False)

# Random Forest Feature Importance
if 'Random Forest' in trained_models:
    rf_model = trained_models['Random Forest']
    rf_importance = rf_model.feature_importances_

    rf_feature_importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': rf_importance
    }).sort_values('Importance', ascending=False)

    print("\n📊 Top 15 Feature Importances (Random Forest):")
    print(rf_feature_importance_df.head(15))
    rf_feature_importance_df.to_csv('rf_feature_importance.csv', index=False)

# Gradient Boosting Feature Importance
if 'Gradient Boosting' in trained_models:
    gb_model = trained_models['Gradient Boosting']
    gb_importance = gb_model.feature_importances_

    gb_feature_importance_df = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': gb_importance
    }).sort_values('Importance', ascending=False)

    print("\n📊 Top 15 Feature Importances (Gradient Boosting):")
    print(gb_feature_importance_df.head(15))
    gb_feature_importance_df.to_csv('gb_feature_importance.csv', index=False)

# สร้างไฟล์รายงาน Excel
print("\n" + "=" * 80)
print("📊 การสร้างไฟล์รายงาน Excel")
print("=" * 80)

print("กำลังสร้างรายงานแบบครบถ้วน...")
complete_report_file = create_complete_excel_report(
    train_df, test_df, X_train, X_test,
    results_df, xgb_feature_importance_df, None, best_model_name
)

print("✅ สร้างไฟล์รายงานเรียบร้อยแล้ว")

# ดาวน์โหลดไฟล์ทั้งหมด
print("\n📥 กำลังดาวน์โหลดไฟล์...")
files_to_download = [
    'stylometric_features_train.csv',
    'stylometric_features_test.csv',
    'stylometric_features_train_with_labels.csv',
    'stylometric_features_test_with_labels.csv',
    'model_performance_results.csv',
    complete_report_file
]

# เพิ่มไฟล์ feature importance
feature_importance_files = ['xgb_feature_importance.csv', 'rf_feature_importance.csv', 'gb_feature_importance.csv']
for file in feature_importance_files:
    if os.path.exists(file):
        files_to_download.append(file)

for file in files_to_download:
    if os.path.exists(file):
        files.download(file)
        print(f"  ✅ ดาวน์โหลด {file}")

# สรุปผลลัพธ์
print("\n" + "=" * 80)
print("🎯 สรุปผลการทดลอง")
print("=" * 80)
print(f"🏆 โมเดลที่ดีที่สุด: {best_model_name}")
print(f"📊 ความแม่นยำสูงสุด: {best_accuracy:.4f}")
print(f"🔢 จำนวน features ที่ใช้: {X_train.shape[1]} features")
print(f"🔢 จำนวนโมเดลที่ทดสอบ: {len(models)} โมเดล")
print(f"📁 จำนวนข้อมูลฝึก: {X_train.shape[0]}")
print(f"📁 จำนวนข้อมูลทดสอบ: {X_test.shape[0]}")
print(f"🏷️  คลาส: {class_names.tolist()}")

print(f"\n💾 ไฟล์ที่สร้าง:")
print("  ✅ stylometric_features_train.csv - Features จากข้อมูลฝึก")
print("  ✅ stylometric_features_test.csv - Features จากข้อมูลทดสอบ")
print("  ✅ stylometric_features_train_with_labels.csv - Features + Labels จากข้อมูลฝึก")
print("  ✅ stylometric_features_test_with_labels.csv - Features + Labels จากข้อมูลทดสอบ")
print("  ✅ model_performance_results.csv - ผลลัพธ์โมเดลทั้งหมด")
print(f"  ✅ {complete_report_file} - รายงานแบบครบถ้วน")

print(f"\n🔍 Features ใหม่ 5 ตัวที่เพิ่ม:")
print("  1. Emotional Tone Score - คะแนนโทนอารมณ์")
print("  2. Formality Index - ดัชนีความเป็นทางการ")
print("  3. Legal Terminology Density - ความหนาแน่นคำทางกฎหมาย")
print("  4. Sentence Structure Complexity - ความซับซ้อนโครงสร้างประโยค")
print("  5. Question Mark Density - ความหนาแน่นเครื่องหมายคำถาม")

print(f"\n🤖 โมเดลใหม่ 6 ตัวที่เพิ่ม:")
print("  1. L1 Regularization (SGD)")
print("  2. L2 Regularization (SGD)")
print("  3. Elastic Net (SGD)")
print("  4. K-Nearest Neighbors")
print("  5. Decision Tree")
print("  6. Neural Network (MLP)")
print("  7. Gradient Boosting")

print("\n🎉 การทดลองเสร็จสมบูรณ์! สามารถดาวน์โหลดไฟล์ทั้งหมดได้แล้ว")

📦 ติดตั้ง packages ที่จำเป็น...
✅ xlsxwriter ติดตั้งแล้ว
✅ textstat ติดตั้งแล้ว
✅ scikit-learn models ติดตั้งแล้ว
✅ ติดตั้ง packages เสร็จสิ้น


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


📁 กรุณาอัพโหลดไฟล์ Train.csv และ Test.csv


Saving Test.csv to Test (1).csv
Saving Train.csv to Train (1).csv

📂 ไฟล์ที่อัพโหลด:
  - Test (1).csv
  - Train (1).csv

🔍 ใช้ไฟล์:
  Train: Train (1).csv
  Test: Test (1).csv

📊 ข้อมูลพื้นฐานของ Dataset
Train dataset shape: (100, 3)
Test dataset shape: (26, 3)

คอลัมน์ใน Train dataset:
['Subject', 'Body', 'Label']

🔍 การตรวจสอบโครงสร้างข้อมูล
มีคอลัมน์ Subject: True
มีคอลัมน์ Body: True
มีคอลัมน์ Label: True
✅ สร้าง combined_text จาก Subject และ Body

🔧 การ Extract Stylometric Features (65 features)
กำลัง extract features จาก train dataset...
Processed 0/100 emails...
Processed 10/100 emails...
Processed 20/100 emails...
Processed 30/100 emails...
Processed 40/100 emails...
Processed 50/100 emails...
Processed 60/100 emails...
Processed 70/100 emails...
Processed 80/100 emails...
Processed 90/100 emails...
กำลัง extract features จาก test dataset...
Processed 0/26 emails...
Processed 10/26 emails...
Processed 20/26 emails...

✅ การ extract features เสร็จสมบูรณ์
จำนวน features ที่ได้: 6

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [09:07:28] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


...

<IPython.core.display.Javascript object>

  ✅ ดาวน์โหลด rf_feature_importance.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  ✅ ดาวน์โหลด gb_feature_importance.csv

🎯 สรุปผลการทดลอง
🏆 โมเดลที่ดีที่สุด: Random Forest
📊 ความแม่นยำสูงสุด: 0.9231
🔢 จำนวน features ที่ใช้: 66 features
🔢 จำนวนโมเดลที่ทดสอบ: 11 โมเดล
📁 จำนวนข้อมูลฝึก: 100
📁 จำนวนข้อมูลทดสอบ: 26
🏷️  คลาส: ['Legitimate', 'Phishing']

💾 ไฟล์ที่สร้าง:
  ✅ stylometric_features_train.csv - Features จากข้อมูลฝึก
  ✅ stylometric_features_test.csv - Features จากข้อมูลทดสอบ
  ✅ stylometric_features_train_with_labels.csv - Features + Labels จากข้อมูลฝึก
  ✅ stylometric_features_test_with_labels.csv - Features + Labels จากข้อมูลทดสอบ
  ✅ model_performance_results.csv - ผลลัพธ์โมเดลทั้งหมด
  ✅ complete_stylometric_analysis_report_20251123_090729.xlsx - รายงานแบบครบถ้วน

🔍 Features ใหม่ 5 ตัวที่เพิ่ม:
  1. Emotional Tone Score - คะแนนโทนอารมณ์
  2. Formality Index - ดัชนีความเป็นทางการ
  3. Legal Terminology Density - ความหนาแน่นคำทางกฎหมาย
  4. Sentence Structure Complexity - ความซับซ้อนโครงสร้างประโยค
  5. Question Mark Density - ความหนาแน่นเครื่องหมายคำถาม

🤖